In [ ]:
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.datavalidation import DataValidation


#Leitura da planilha
try:
    df = pd.read_excel('modelo02.xlsx')
except FileNotFoundError:
    print('Arquivo modelo02.xlsx não encontrado!')
    exit()



departamentos = df['Departamento'].unique()



# Configurações das validações de dados
validations = {
    'EXISTE MONITORAMENTO?': {'type': 'list', 'source': ['SIM', 'NÃO']},
    'SEVERIDADE/ABRANGENCIA': {'type': 'list', 'source': ['BAIXA', 'MÉDIA', 'ALTA']},
    'PERIODICIDADE': {'type': 'list', 'source': ['NULA', 'DIÁRIA', 'SEMANAL', 'MENSAL', 'SEMESTRAL', 'ANUAL']}
}





# Loop pelos departamentos
for departamento in departamentos:
    # Filtra os dados por departamento
    departamento_df = df[df['Departamento'] == departamento]
    
    # Adiciona as novas colunas ao dataframe
    novas_colunas_df = pd.DataFrame({
        'SEVERIDADE/ABRANGENCIA': [''] * len(departamento_df),
        'EXISTE MONITORAMENTO?': [''] * len(departamento_df),
        'QUEM MONITORA?': '',
        'PERIODICIDADE': [''] * len(departamento_df)
    }, index=departamento_df.index)

    departamento_df = pd.concat([departamento_df, novas_colunas_df], axis=1)

    # Cria um nome de arquivo a partir do nome do departamento
    nome_arquivo = f'{departamento}.xlsx'

    # Salva o arquivo excel
    with pd.ExcelWriter(nome_arquivo, engine='openpyxl') as writer:
        departamento_df.to_excel(writer, index=False, sheet_name='Sheet1')
        worksheet = writer.sheets['Sheet1']
        
        # Adiciona as validações de dados
        for col_name, validation in validations.items():
            dv = DataValidation(type=validation['type'], formula1=f'"{",".join(validation["source"])}"')
            coluna = get_column_letter(departamento_df.columns.get_loc(col_name) + 1)
            dv.add(f'{coluna}2:{coluna}{len(departamento_df) + 1}')
            worksheet.add_data_validation(dv)

        # Ajusta a largura das colunas
        for i, coluna in enumerate(departamento_df.columns):
            largura = 25
            letra_coluna = get_column_letter(i+1)
            worksheet.column_dimensions[letra_coluna].width = largura
    
    print(f'{nome_arquivo} salvo com sucesso!')